This notebook scrapes satellite images for each leak repair. For each location it gets a NxM rectangle around the leak before and after it was repaired. Then it collated all the data into h5 files and all the metadata into json files.

It takes days to run because of rate limiting on the google earth api. Because of limited satelite coverage you might find matches for only 10% of the leaks.

## Modifying

- make sure google earth is setup
- load leaks, so they pass the asserts
- change params
- run rest of cells

In [1]:
from path import Path
import arrow
import json
import pytz
from pprint import pprint
from tqdm import tqdm
import re, os, collections, itertools, uuid, logging
import tempfile
import tables

import zipfile
import urllib

import ee
import pyproj
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (15, 5) # bigger plots
plt.style.use('fivethirtyeight')
%matplotlib inline
%precision 4

'%.4f'

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
helper_dir = str(Path('..').abspath())
if helper_dir not in os.sys.path:
    os.sys.path.append(helper_dir)
    
from leak_helpers.earth_engine import display_ee, get_boundary, tifs2np, bands_s2, download_image, bands_s2, bands_s1, bands_l7, bands_l8

In [4]:

crs_grid = 3857
notebook_name='testing_earth_engine-l7-AUTX'
ts=arrow.utcnow().format('YYYYMMDD-HH-mm-ss')
data_dir = Path('../../data/').abspath()
bands = bands_l7

# since the lowest res band is 60m and I want to capture neighbours I should get 6+ pixels
pixel_length = 25.0
resolution_min = 15.0 # m
time_bin_delta = 60*60*24*28 # how long before a leak to look (in seconds)
# TODO get closest but let me filter for time

# init
temp_dir = Path(tempfile.mkdtemp(prefix=notebook_name+'-', suffix='-'+ts)).abspath()
# output_dir = data_dir.joinpath('{ts:}_{notebook_name:}'.format(ts=ts,notebook_name=notebook_name))
output_dir = Path('../../data/scraped_satellite_images/20170314-05-26-52_testing_earth_engine-l7-AUTX_v2').abspath()
cache_dir = output_dir.joinpath('ee_l7_AUTX-leaks_cache_v2').abspath()

output_dir.makedirs_p()
temp_dir.makedirs_p()
cache_dir.makedirs_p()

logger = logging.getLogger(notebook_name)
logger.setLevel(logging.WARN)

crs_grid_proj = pyproj.Proj('epsg:%s'%crs_grid)

temp_dir, output_dir, cache_dir

(Path('/tmp/testing_earth_engine-l7-AUTX-9haczdn9-20230511-16-22-56'),
 Path('/home/dmitrii/Documents/work/satellite_leak_detection/data/scraped_satellite_images/20170314-05-26-52_testing_earth_engine-l7-AUTX_v2'),
 Path('/home/dmitrii/Documents/work/satellite_leak_detection/data/scraped_satellite_images/20170314-05-26-52_testing_earth_engine-l7-AUTX_v2/ee_l7_AUTX-leaks_cache_v2'))

In [5]:

metadata_file = output_dir.joinpath('script_metadata.json')

# write metadata to json
metadata = dict(
    pixel_length=pixel_length,
    resolution_min=resolution_min,
    bands=bands,
    ts=ts,
    notebook_name=notebook_name,
    crs_grid=crs_grid,
    cache_dir=str(cache_dir),
    temp_dir=str(temp_dir),
    output_dir=str(output_dir),
)
json.dump(metadata, open(metadata_file,'w'))

                                    

# earth engine

Setup instructions here
- first need to apply for an account and wait ~ 1day
- https://developers.google.com/earth-engine/python_install#setting-up-authentication-credentials

Refs:
- api https://developers.google.com/earth-engine/
- code examples https://code.earthengine.google.com/
- sentinel1 https://developers.google.com/earth-engine/sentinel1
    - `ee.ImageCollection('COPERNICUS/S2_GRD');`
    - `ee.ImageCollection('COPERNICUS/S1_GRD');`
- keras and google earth https://github.com/patrick-dd/landsat-landstats

In [6]:
ee.Initialize() # should give no errors, if so follow instructions


# test
image = ee.Image('srtm90_v4')
# assert image.getInfo()=={'type': 'Image', 'properties': {'system:time_start': 950227200000, 'system:asset_size': 18827626666, 'system:time_end': 951177600000}, 'bands': [{'data_type': {'type': 'PixelType', 'max': 32767, 'min': -32768, 'precision': 'int'}, 'crs': 'EPSG:4326', 'id': 'elevation', 'dimensions': [432000, 144000], 'crs_transform': [0.000833333333333, 0.0, -180.0, 0.0, -0.000833333333333, 60.0]}], 'id': 'srtm90_v4', 'version': 1463778555689000}
# print('ok')

# ee.Geometry.Point([117.21079620254062, -30.94712385398404])

In [7]:
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [432000, 144000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.0008, 0, -180, 0, -0.0008, 60]}],
 'version': 1494271934303000.0000,
 'id': 'srtm90_v4',
 'properties': {'system:time_start': 950227200000,
  'system:time_end': 951177600000,
  'system:asset_size': 18827626666}}

In [8]:
boundary = ee.Geometry.Rectangle([-122.2806, 37.1209, -122.0554, 37.2413])


In [9]:
boundary = ee.Geometry.Rectangle([30.24167, -97.76019, 30.24367, -97.75719])

In [10]:
boundary

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              30.24167,
              -97.75719
            ],
            [
              30.24167,
              -97.76019
            ],
            [
              30.24367,
              -97.76019
            ],
            [
              30.24367,
              -97.75719
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [11]:
collection = ee.ImageCollection('LANDSAT/LE7_L1T')\
    .filterBounds(boundary)\
    .filterDate(933828614605,1488776737937)\
    .sort('system:time_start', opt_ascending=False)
#          .filterDate(933828614605,1488776737937)\
         

In [12]:
collection.first().getInfo()

# Load leaks

In [13]:
# load wa leaks
leaks_ATX = gpd.read_file(data_dir.joinpath('leak_datasets/austin_leaks/derived/austin_leaks-repairs.geojson'))


# they have to be after launch
s3_launch_ts=pd.Timestamp('1 Jan 1999', tz='UTC')
leaks_ATX = leaks_ATX[pd.to_datetime(leaks_ATX.COMPDTTM)>=s3_launch_ts]
leaks_ATX['REPO_Date']=leaks_ATX['COMPDTTM']
leaks_ATX['leak_id']=leaks_ATX.OBJECTID.apply(lambda x:'ATX-%s'%x)
# leaks=leaks_ATX
# leaks.index = leaks.leak_id
len(leaks_ATX)

23131

In [14]:
# choose one leak for now
leak = leaks_ATX.sample()
leak

,id,leak_id,REPO_Date,22,ADDRKEY,CITY,COMPDTTM,DESCRIPT,FullStreetName,INITDTTM,...,OBJECTID,PREDIR,QTYCALLS,STNAME,STNO,STSUB,SUFFIX,WONO,ZIP,geometry
11856,59342,ATX-59342,2011-08-15 02:30:00+00:00,934188.0,367779.0,DEL VALLE,2011-08-15 02:30:00+00:00,WATER MAIN LEAK,14800 JACOBSON RD,2011-08-14 16:28:00+00:00,...,59342,,20,JACOBSON,14800,,RD,962077.0,78617-,POINT (-97.61222 30.13449)


# Fetching sentinal-1 and sentinel 2 images

For a leak repair, grab the image before and after it

Note roughly 10% have results for a 1 day temporal bin

In [15]:
def get_cached_ids():
    cache_dirs = [str(f.relpath(cache_dir)).split('_')[0] for f in cache_dir.listdir()]
    return cache_dirs

def init_cache(leak_id):
    """We will cache downloads in folders like 'id_after'"""
    if leak_id:
        cache_subdir = cache_dir.joinpath(leak_id+'_after')
        cache_subdir.makedirs_p()
        cache_subdir = cache_dir.joinpath(leak_id+'_before')
        cache_subdir.makedirs_p()
    return get_cached_ids()

In [16]:
for i in [10,50,100,1000,2000]:
    leak=leaks_ATX.iloc[[i]]
    leak_id = str(leak.OBJECTID.values[0])
    print(get_boundary(leak))

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -97.77590634688022,
              30.258385434260425
            ],
            [
              -97.77590634688022,
              30.25683356120609
            ],
            [
              -97.77410971631198,
              30.25683356120609
            ],
            [
              -97.77410971631198,
              30.258385434260425
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})
ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -97.7631394695563,
              30.157102635295765
            ],
            [
              -97.7631394695563,
              30.155

For each point
- find the nearest image before the repair
- and the soonest image after repair
- save a part of each with metadata

Later we can filter, interpolate, and read into numpy arrays

In [17]:
distance = resolution_min*(pixel_length/2.0-0.5)

In [18]:
# test with one image
for i in [10,50,100,1000,2000]:
    leak=leaks_ATX.iloc[[i]]
    leak_id = str(leak.OBJECTID.values[0])

    repo_date_ts = arrow.get(str(leak.REPO_Date.values[0])).timestamp
    boundary = get_boundary(leak, distance=distance)
    sentinel2_before = ee.ImageCollection('LANDSAT/LE7_L1T')\
        .filterBounds(boundary)\
        .filterDate(933828614605,1488776737937)\
        .sort('system:time_start', opt_ascending=False) # first will be latest
    image = sentinel2_before.first().clip(boundary)
    name=leak_id+'_after'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    data = tifs2np(path,files,bands=bands_l7)
    print(i, [(d.shape,d.sum()) for d in data])
    for d in data:
        assert d.shape[0]==pixel_length, 'the downloaded image is the wrong size, tweak distance'
        assert d.shape[1]==pixel_length
    assert np.sum(data)!=0,'should not be empty (make sure you are using the right bands)'

10 [((25, 25), 163.18039215686272), ((25, 25), 126.6235294117647), ((25, 25), 117.31372549019608), ((25, 25), 121.85882352941177), ((25, 25), 157.54901960784315), ((25, 25), 342.1803921568627), ((25, 25), 404.721568627451), ((25, 25), 107.26274509803922), ((25, 25), 103.35686274509806)]
50 [((25, 25), 155.77647058823527), ((25, 25), 129.42745098039217), ((25, 25), 119.1058823529412), ((25, 25), 156.87843137254902), ((25, 25), 217.1686274509804), ((25, 25), 333.52156862745096), ((25, 25), 388.7254901960784), ((25, 25), 134.17254901960786), ((25, 25), 117.23529411764704)]
100 [((25, 25), 185.45098039215685), ((25, 25), 152.83529411764707), ((25, 25), 155.07450980392156), ((25, 25), 119.63529411764706), ((25, 25), 178.4470588235294), ((25, 25), 358.2039215686274), ((25, 25), 433.41176470588243), ((25, 25), 134.67843137254903), ((25, 25), 113.10588235294118)]
1000 [((25, 25), 168.93333333333334), ((25, 25), 148.33333333333331), ((25, 25), 159.70588235294116), ((25, 25), 120.08627450980391)

In [11]:
import time 
cached_ids = get_cached_ids()

def get_image_for_leak(i, cached_ids=cached_ids):    
    leak = leaks_ATX.iloc[[i]]
    repo_date_ts = arrow.get(leak.REPO_Date.values[0]).timestamp
#     distance = resolution_min*(pixel_length/2.0+1)
    
    
    # crappy way or recording that we tried this one
    leak_id = str(leak.OBJECTID.values[0])
    if leak_id in cached_ids:
        logger.info('Skipping cached download for leak id %s ',leak_id)
        return
    
    boundary = get_boundary(leak, distance=distance)
    
    # get image day before    
    sentinel2_before = ee.ImageCollection('LANDSAT/LE7_L1T')\
        .filterBounds(boundary)\
        .filterDate((repo_date_ts-time_bin_delta)*1000,(repo_date_ts)*1000)\
        .sort('system:time_start', opt_ascending=False) # first will be latest
    
    results = sentinel2_before.size().getInfo()
    if results<1:
        logger.info('Error no results for day before %s',leak_id)
        cached_ids = init_cache(leak_id) # so we know there where no results
        return
        
    # get image day after
    sentinel2_after = ee.ImageCollection('LANDSAT/LE7_L1T')\
        .filterBounds(boundary)\
        .filterDate((repo_date_ts)*1000,(repo_date_ts+time_bin_delta*6)*1000)\
        .sort('system:time_start', opt_ascending=True) # first will be earliest
        
    results = sentinel2_after.size().getInfo()
    if results<1:
        logger.info('Error no results for day after, id %s',leak_id)
        cached_ids = init_cache(leak_id) # so we know there where no results
        return
        
    # download as save images    
    logger.info('results for %s', leak_id)
    image = ee.Image(sentinel2_before.first()).clip(boundary)
    name=leak_id+'_before'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    # also save metadata so we can filter by date
    with open(path.joinpath('metadata.json'), 'w') as fo:
        metadata = dict(
            image=image.getInfo(),
            scale=resolution_min,
            crs=crs_grid,
            name=name,
            distance=distance,
            leak=json.loads(leak.to_json())
        )
        json.dump(metadata, fo)

    image = ee.Image(sentinel2_after.first()).clip(boundary)
    name=leak_id+'_after'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    with open(path.joinpath('metadata.json'), 'w') as fo:
        metadata = dict(
            image=image.getInfo(),
            scale=resolution_min,
            crs=crs_grid,
            name=name,
            distance=distance,
            leak=json.loads(leak.to_json())
        )
        json.dump(metadata, fo)
        
for i in tqdm(range(len(leaks_ATX))):
    try:
        get_image_for_leak(i)
    except urllib.error.HTTPError as e:
        print(i,e)
        if e.code == 429:
             time.sleep(13);
    except Exception as e:
        print(i,e)
        ee.Initialize()

17868 Earth Engine memory capacity exceeded.
17869 Earth Engine memory capacity exceeded.
17940 Earth Engine memory capacity exceeded.
17941 Earth Engine memory capacity exceeded.
17945 Earth Engine memory capacity exceeded.
17956 Earth Engine memory capacity exceeded.
17958 Earth Engine memory capacity exceeded.
17967 Earth Engine memory capacity exceeded.
17971 Earth Engine memory capacity exceeded.
17999 Earth Engine memory capacity exceeded.
18012 Earth Engine memory capacity exceeded.



# parsing tiffs

In [20]:
# This loads it as X and y for machine learning, and also time and metadata so we can filter
import shapely
X = []
y = []
t = []
m = []
discarded=[]
cdirs = [cdir for cdir in cache_dir.listdir() if ('_after_' in cdir) or ('_before_' in cdir)]
for path in tqdm(cdirs):
    if not path.isdir(): continue
    files = [file.relpath(path) for file in path.listdir() if file.isfile() and file.endswith('.tif')]
    if files:
        # check metadata
        try:
            metadata = json.load(open(path.joinpath('metadata.json')))
        except (FileNotFoundError, ValueError) as e:
            path.move(path.dirname().dirname().joinpath('.deleteme-'+str(uuid.uuid4())))
            if '_after_' in path: # also delete the before path                
                path_after = Path(path.replace('_after_','_before_'))
                if path_after.isdir():
                    path_after.move(path.dirname().dirname().joinpath('.deleteme-'+str(uuid.uuid4())))
            logger.error('Invalid metadata.json, deleted folder %s, please rerun scraping cell to rescrape this image', path)
            continue
        
        # e.g. lets filter it so "before" image are only 1 day before
        if '_before_' in path.basename():
            yy = True
        else:
            yy = False
        
        # work out time gap too
        t1 = arrow.get(metadata['image']['properties']['system:time_end']/1000)
        t0 = arrow.get(metadata['leak']['features'][0]['properties']['REPO_Date'])
        td=t1-t0
        tt = td.total_seconds()
        
        # load data
        data = tifs2np(path,files,bands=bands)
             
        # check we don't have empty bands 1-13
        empty_bands = np.array([d.sum() for d in data])==0
        
        # lets check we didn't get the edge of an image
        bbox = np.array(metadata['image']['properties']['system:footprint']['coordinates'][0])
        loc = metadata['leak']['features'][0]['geometry']['coordinates']
        minx=bbox[:,0].min()
        maxx=bbox[:,0].max()
        miny=bbox[:,1].min()
        maxy=bbox[:,1].max()
        bbox_shp = shapely.geometry.box(
            minx=minx,
            maxx=maxx,
            miny=miny,
            maxy=maxy
        )
        loc_shp = shapely.geometry.Point(loc[0],loc[1])
        shapely.geometry.GeometryCollection([bbox_shp, loc_shp])
        try:
            assert loc_shp.intersects(bbox_shp), 'leak location should be inside image'
            assert bbox_shp.centroid.almost_equals(loc_shp, decimal=5), 'leak should be near center of image'
            assert (np.array([d.shape for d in data])==pixel_length).all(), 'image area should be the right amount of pixels'
            assert (maxx-minx)/(maxy-miny)<1.3, 'should be roughly square'
            assert (maxx-minx)/(maxy-miny)>0.7, 'should be roughly square'
            assert not empty_bands.all(), 'should not have all bands empty'
        except Exception as exc:
            print(path, exc)
#             raise(exc)
            discarded.append(path)
        else:
            X.append(data)
            y.append(yy)
            t.append(tt)
            m.append(metadata)

len(X), len(discarded)

Invalid metadata.json, deleted folder ../data/20170314-05-26-52_testing_earth_engine-l7-AUTX_v2/ee_l7_AUTX-leaks_cache_v2/65498_before_3857_15.0, please rerun scraping cell to rescrape this image


(40745, 0)

In [21]:
path

Path('../data/20170314-05-26-52_testing_earth_engine-l7-AUTX_v2/ee_l7_AUTX-leaks_cache_v2/70616_before_3857_15.0')

In [22]:
# shuffle
from sklearn.utils import shuffle
X,y,m= shuffle(X,y,m,random_state=1337)

In [ ]:
# which bands do we have?
a=np.array([x.sum(-1).sum(-1)==0 for x in X])
print('amount of each band',list(zip(bands,a.sum(0))))
print('mean amount of bands',a.sum(1).mean())

amount of each band [('B1', 0), ('B2', 0), ('B3', 0), ('B4', 0), ('B5', 0), ('B6_VCID_1', 0), ('B6_VCID_2', 0), ('B7', 0), ('B8', 0)]
mean amount of bands 0.0


In [ ]:
# save using hdf5 (so keras can easily load it) and json 
import h5py
h5file = output_dir.joinpath('data.h5')
with h5py.File(h5file, 'w') as h5f:
    h5f.create_dataset('X', data=X)
    h5f.create_dataset('y', data=y)

json.dump(m,open(output_dir.joinpath('data_metadata.json'),'w'))

with open(output_dir.joinpath('readme.md'),'w') as fo:
    fo. write("""
Files:
- ee_S1_AUTX-leaks_cache- cached tiff files
- script_metadata.json - information on scraping script
- data.h5 contains X, y, and t.
    - X: tiff files for each band loaded into an array of shape (Leak, Bands, width, length)
    - y: True for before the leak, False for after
- data_metadata: array of metadata for each leak in X. Each contain info on leak, image, and image search
    
Loading: 
```py
# load
metadatas = json.load(open('data_metadata.json'))
with h5py.File('data.h5','r') as h5f:
    X2 = h5f['X'][:]
    y2 = h5f['y'][:]
y
```
    """)

In [ ]:
# test load
metadatas = json.load(open(output_dir.joinpath('data_metadata.json')))
with h5py.File(output_dir.joinpath('data.h5'),'r') as h5f:
    X2 = h5f['X'][:]
    y2 = h5f['y'][:]
X2.shape, y2, metadatas[0].keys()